In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import boto3
import io
import pandas as pd
from settings import *
import joblib
from awstk import *

s3_client = boto3.client('s3', **credentials)

body = s3_client.get_object(Bucket=bucket_name, Key=iris_data_key)['Body']
buffer = io.BytesIO(body.read())
df = pd.read_parquet(buffer)


In [2]:
x = df.iloc[:,:4]
y = df.iloc[:,-1:]
assert len(df.columns) == len(x.columns) + len(y.columns)

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_classifier.fit(x_train, y_train)
y_pred = rf_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy={accuracy}')
assert accuracy > 0.7

accuracy=1.0


/Users/alexlewzey/repository/example_project/.venv/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [4]:
buffer = io.BytesIO()
joblib.dump(rf_classifier, buffer)
buffer.seek(0)
response = s3_client.put_object(Body=buffer, Bucket=bucket_name, Key=model_key)
assert_response_eq_200(response)

In [6]:
body = s3_client.get_object(Bucket=bucket_name, Key=model_key)['Body']
rf_classifier_test = joblib.load(io.BytesIO(body.read()))
assert hasattr(rf_classifier_test, 'classes_')
assert isinstance(rf_classifier_test, RandomForestClassifier)